In [1]:
#import dependencies 
import pandas as pd
import geopandas as gpd
import numpy
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
#load in file
file_to_load = "Resources/chicago_crimes.csv"

#store into pandas data frame
Crime_Data = pd.read_csv(file_to_load)

In [3]:
Crime_Data.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
0,JE145602,12/22/2020 08:59:00 PM,004XX W 58TH ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,N,N,711,20.0,11,NaN,NaN,NaN,NaN,NaN
1,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)"
2,JE145783,04/01/2020 12:00:00 PM,027XX S ARCHER AVE,0820,THEFT,$500 AND UNDER,STREET,N,N,913,11.0,06,NaN,NaN,NaN,NaN,NaN
3,JD390623,09/30/2020 12:00:00 PM,018XX N Sedgwick St,0810,THEFT,OVER $500,RESIDENCE,N,N,1814,43.0,06,1173303.0,1912547.0,41.915454,-87.638727,"(41.915454333, -87.638727274)"
4,JD191383,03/19/2020 10:00:00 AM,051XX N MILWAUKEE AVE,1585,SEX OFFENSE,OTHER,POLICE FACILITY / VEHICLE PARKING LOT,N,N,1623,45.0,17,1138424.0,1933673.0,41.974130,-87.766357,"(41.974129858, -87.766357256)"


In [4]:
Crime_Data.dropna(inplace = True)

In [5]:
Crime_Data.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION
1,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)"
3,JD390623,09/30/2020 12:00:00 PM,018XX N Sedgwick St,0810,THEFT,OVER $500,RESIDENCE,N,N,1814,43.0,06,1173303.0,1912547.0,41.915454,-87.638727,"(41.915454333, -87.638727274)"
4,JD191383,03/19/2020 10:00:00 AM,051XX N MILWAUKEE AVE,1585,SEX OFFENSE,OTHER,POLICE FACILITY / VEHICLE PARKING LOT,N,N,1623,45.0,17,1138424.0,1933673.0,41.974130,-87.766357,"(41.974129858, -87.766357256)"
5,JD381795,09/13/2020 12:00:00 PM,015XX N KINGSBURY ST,0810,THEFT,OVER $500,GROCERY FOOD STORE,Y,N,1822,27.0,06,1169505.0,1910252.0,41.909240,-87.652748,"(41.909240284, -87.65274758)"
6,JD219381,04/28/2020 07:15:00 PM,085XX S GIVINS CT,1330,CRIMINAL TRESPASS,TO LAND,RESIDENCE - YARD (FRONT / BACK),N,N,622,21.0,26,1172711.0,1848282.0,41.739118,-87.642800,"(41.739118468, -87.642799939)"


In [6]:
Crime_Data["lat_long"]= Crime_Data["LATITUDE"].apply(lambda x:str(x)) +","+ Crime_Data["LONGITUDE"].apply(lambda x:str(x))

In [7]:
Crime_Data.head()

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,lat_long
1,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)","41.796167139,-87.70951943"
3,JD390623,09/30/2020 12:00:00 PM,018XX N Sedgwick St,0810,THEFT,OVER $500,RESIDENCE,N,N,1814,43.0,06,1173303.0,1912547.0,41.915454,-87.638727,"(41.915454333, -87.638727274)","41.915454333,-87.638727274"
4,JD191383,03/19/2020 10:00:00 AM,051XX N MILWAUKEE AVE,1585,SEX OFFENSE,OTHER,POLICE FACILITY / VEHICLE PARKING LOT,N,N,1623,45.0,17,1138424.0,1933673.0,41.974130,-87.766357,"(41.974129858, -87.766357256)","41.974129858,-87.766357256"
5,JD381795,09/13/2020 12:00:00 PM,015XX N KINGSBURY ST,0810,THEFT,OVER $500,GROCERY FOOD STORE,Y,N,1822,27.0,06,1169505.0,1910252.0,41.909240,-87.652748,"(41.909240284, -87.65274758)","41.909240284,-87.65274758"
6,JD219381,04/28/2020 07:15:00 PM,085XX S GIVINS CT,1330,CRIMINAL TRESPASS,TO LAND,RESIDENCE - YARD (FRONT / BACK),N,N,622,21.0,26,1172711.0,1848282.0,41.739118,-87.642800,"(41.739118468, -87.642799939)","41.739118468,-87.642799939"


In [8]:
Crime_Data.groupby('BEAT').first()['lat_long']

BEAT
111     41.883500187,-87.627876698
112     41.881683753,-87.624353027
113     41.877578503,-87.629282088
114     41.887705775,-87.617022196
121      41.879331765,-87.63824099
                   ...            
2531    41.904615737,-87.773022962
2532     41.90834437,-87.763383589
2533    41.904159474,-87.747044455
2534    41.903446578,-87.734399671
2535    41.909907002,-87.724577987
Name: lat_long, Length: 274, dtype: object

In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")

In [13]:
def get_zipcode(lat_long):
    location = geolocator.geocode(lat_long)
    zip_code = location.raw['display_name'].split(",")[-2].strip()
    return zip_code

In [14]:
%%time
beats_zipcodes = Crime_Data.groupby('BEAT').first()['lat_long'].apply(get_zipcode)

CPU times: user 5.55 s, sys: 799 ms, total: 6.34 s
Wall time: 2min 19s


In [15]:
beats_zipcodes.name="Zipcode"

In [16]:
beats_zipcodes

BEAT
111     60602
112     60603
113     60604
114     60601
121     60606
        ...  
2531    60651
2532    60651
2533    60651
2534    60651
2535    60302
Name: Zipcode, Length: 274, dtype: object

In [17]:
Crime_Data.merge(pd.DataFrame(beats_zipcodes).reset_index(),on="BEAT")

,CASE#,DATE OF OCCURRENCE,BLOCK,IUCR,PRIMARY DESCRIPTION,SECONDARY DESCRIPTION,LOCATION DESCRIPTION,ARREST,DOMESTIC,BEAT,WARD,FBI CD,X COORDINATE,Y COORDINATE,LATITUDE,LONGITUDE,LOCATION,lat_long,Zipcode
0,JD220983,04/28/2020 04:00:00 PM,034XX W 53RD PL,0610,BURGLARY,FORCIBLE ENTRY,RESIDENCE,N,N,822,14.0,05,1154351.0,1868931.0,41.796167,-87.709519,"(41.796167139, -87.70951943)","41.796167139,-87.70951943",60632
1,JD368147,05/06/2020 07:00:00 PM,035XX W 57TH ST,0820,THEFT,$500 AND UNDER,APARTMENT,N,N,822,23.0,06,1153830.0,1866591.0,41.789756,-87.711492,"(41.789756197, -87.711492062)","41.789756197,-87.711492062",60632
2,JD455983,12/11/2020 06:00:00 AM,056XX S SAWYER AVE,0820,THEFT,$500 AND UNDER,STREET,N,N,822,14.0,06,1155678.0,1867012.0,41.790875,-87.704705,"(41.790874602, -87.704704643)","41.790874602,-87.704704643",60632
3,JD364284,09/11/2020 04:20:00 PM,052XX S LAWNDALE AVE,0485,BATTERY,AGGRAVATED OF A CHILD,RESIDENCE,N,N,822,23.0,04B,1152596.0,1869582.0,41.797988,-87.715938,"(41.797988346, -87.71593806)","41.797988346,-87.71593806",60632
4,JD364146,09/11/2020 05:00:00 PM,032XX W 55TH ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,STREET,N,Y,822,14.0,08B,1155567.0,1867964.0,41.793489,-87.705086,"(41.793489253, -87.705086148)","41.793489253,-87.705086148",60632
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196596,JD472278,12/29/2020 07:43:00 AM,0000X W TERMINAL ST,1350,CRIMINAL TRESPASS,TO STATE SUP LAND,AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA,Y,N,1652,41.0,26,1100726.0,1934289.0,41.976421,-87.904976,"(41.9764212, -87.904976266)","41.9764212,-87.904976266",60124
196597,JE111608,01/12/2021 04:40:00 PM,0000X W TERMINAL ST,0850,THEFT,ATTEMPT THEFT,AIRPORT TERMINAL UPPER LEVEL - SECURE AREA,N,N,1652,41.0,06,1100726.0,1934289.0,41.976421,-87.904976,"(41.9764212, -87.904976266)","41.9764212,-87.904976266",60124
196598,JE110382,01/12/2021 08:35:00 AM,0000X W TERMINAL ST,0860,THEFT,RETAIL THEFT,AIRPORT TERMINAL UPPER LEVEL - SECURE AREA,N,N,1652,41.0,06,1100726.0,1934289.0,41.976421,-87.904976,"(41.9764212, -87.904976266)","41.9764212,-87.904976266",60124
196599,JD471011,12/27/2020 06:00:00 PM,001XX W OHARE ST,0910,MOTOR VEHICLE THEFT,AUTOMOBILE,AIRPORT EXTERIOR - NON-SECURE AREA,N,N,1652,41.0,07,1100332.0,1934628.0,41.977357,-87.906419,"(41.977356862, -87.906419005)","41.977356862,-87.906419005",60124
